In [1]:
import sys
import os
import string
from glob import glob

# Create by index of symbol folder in the train and test folders

In [2]:
ALL_SYMBOLS = string.ascii_uppercase# + string.digits

In [3]:
target_mapper = {}
for index, value in enumerate(ALL_SYMBOLS):
    target_mapper[value] = index

In [4]:
target_mapper["A"], target_mapper["Z"]

(0, 25)

In [5]:
target_mapper

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [6]:
parent_dir = os.getcwd()

In [7]:
train_data = os.path.join(parent_dir, "training_data")
test_data = os.path.join(parent_dir, "test_data")

In [8]:
for symbol in target_mapper.keys():
    path = os.path.join(train_data, str(target_mapper[symbol]))
    try:
        os.mkdir(path)
    except:
        print(f"{path} already exists")

    path = os.path.join(test_data, str(target_mapper[symbol]))
    try:
        os.mkdir(path)
    except:
        print(f"{path} already exists")

E:\projects\captcha_solver\training_data\0 already exists
E:\projects\captcha_solver\test_data\0 already exists
E:\projects\captcha_solver\training_data\1 already exists
E:\projects\captcha_solver\test_data\1 already exists
E:\projects\captcha_solver\training_data\2 already exists
E:\projects\captcha_solver\test_data\2 already exists
E:\projects\captcha_solver\training_data\3 already exists
E:\projects\captcha_solver\test_data\3 already exists
E:\projects\captcha_solver\training_data\4 already exists
E:\projects\captcha_solver\test_data\4 already exists
E:\projects\captcha_solver\training_data\5 already exists
E:\projects\captcha_solver\test_data\5 already exists
E:\projects\captcha_solver\training_data\6 already exists
E:\projects\captcha_solver\test_data\6 already exists
E:\projects\captcha_solver\training_data\7 already exists
E:\projects\captcha_solver\test_data\7 already exists
E:\projects\captcha_solver\training_data\8 already exists
E:\projects\captcha_solver\test_data\8 already

# Split every image, generated by captcha into the either training folder or testing

In [9]:
import cv2 as cv
import random
import re
import numpy as np

In [10]:
all_captchas = glob("captcha_images/*.png")

In [11]:
sampleList = ["training_data", "test_data"]
 
randomList = random.choices(sampleList, weights=(80, 20), k=len(all_captchas))

In [12]:
def get_min(val1, val2):
    res = val2
    if val1 < val2:
        res = val1
    return res

def get_max(val1, val2):
    res = val2
    if val1 > val2:
        res = val1
    return res

In [13]:
def get_corners(contour):
    min_x = contour[0][0][0]
    min_y = contour[0][0][1]
    max_x = contour[0][0][0]
    max_y = contour[0][0][1]
    for point in contour:
        x, y = point[0]
        min_x = get_min(min_x, x)
        min_y = get_min(min_y, y)
        max_x = get_max(max_x, x)
        max_y = get_max(max_y, y)

    return ((min_x, min_y), (max_x, max_y))

In [14]:
def inner_contour(boxB, boxA):
    startA, endA = boxA
    startB, endB = boxB
    start_lower = (startA[0] <= startB[0]) and (startA[1] <= startB[1])
    end_bigger = (endA[0] >= endB[0]) and (endA[1] >= endB[1])
    return start_lower and end_bigger

In [15]:
def filter_inner_contour(boxes):
    result = []
    for i in range(len(boxes)):
        flag = False
        for j in range(len(boxes)):
            if j != i:
                if inner_contour(boxes[i], boxes[j]):
                    flag = True
        if not flag:
            result.append(boxes[i])
    return result

In [16]:
def get_square(start, end):
    length = end[0] - start[0]
    height = end[1] - start[1]
    return length * height

In [17]:
def split_rectangles(boxes):
    boxes = list(sorted(boxes, key=lambda x: x[1][0] - x[0][0]))
    box_to_split = boxes.pop()
    start, end = box_to_split
    width = end[0] - start[0]
    height = end[1] - start[1]
    box1 = ((start[0], start[1]), (start[0] + (width // 2), end[1]))
    box2 = ((start[0] + (width // 2), start[1]), (end[0], end[1]))
    boxes.append(box1)
    boxes.append(box2)
    return boxes

In [18]:
def increase_14px(box):
    start, end = box
    start = (start[0] - 3, start[1] - 3)
    end = (end[0] + 3, end[1] + 3)
    return start, end

In [130]:
def expand_rectangle(rectangle, expansion_factor):
    """
    Expand the coordinates of a rectangle by a given expansion factor.

    Args:
    rectangle (tuple): Tuple containing (x_min, y_min, x_max, y_max) coordinates of the rectangle.
    expansion_factor (float): Factor by which to expand the rectangle.

    Returns:
    tuple: Expanded rectangle coordinates in the format (x_min_expanded, y_min_expanded, x_max_expanded, y_max_expanded).
    """
    start, end = rectangle
    x_min, y_min = start
    x_max, y_max = end
    width = x_max - x_min
    height = y_max - y_min
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2

    new_width = width * expansion_factor
    new_height = height * expansion_factor

    x_min_expanded = max(int(x_center - new_width / 2), 0)
    y_min_expanded = max(int(y_center - new_height / 2), 0)
    x_max_expanded = int(x_center + new_width / 2)
    y_max_expanded = int(y_center + new_height / 2)

    return ((x_min_expanded, y_min_expanded), (x_max_expanded, y_max_expanded))

In [131]:
idx = 65
all_captchas[idx]

'captcha_images\\AFHL.png'

In [132]:
curr_symb = 0
ccc = 0
for idx in range(len(all_captchas)):
    if randomList[idx]=="training_data":
        path = train_data
    else:
        path = test_data
        
    img = cv.imread(all_captchas[idx])
    threshold = 150
    
    # cv.imshow("", img)
    # cv.waitKey()
    # cv.destroyAllWindows()
    
    # convert to grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # blur
    blur = cv.GaussianBlur(gray, (0, 0), sigmaX=33, sigmaY=33)
    # divide
    divide = cv.divide(gray, blur, scale=255)
    # otsu threshold
    thresh = cv.threshold(divide, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)[1] # 
    # apply morphology
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (4, 4))
    morph = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)
    
    canny_output = cv.Canny(morph, threshold, threshold * 2)
    canny_output_cp = canny_output.copy()
    canny_output = cv.blur(canny_output, (3, 3))
    
    # cv.imshow("", canny_output)
    # cv.waitKey()
    # cv.destroyAllWindows()
    
    contours, hierarchy = cv.findContours(canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = [get_corners(i) for i in contours]
    contours = filter_inner_contour(contours)
    contours = sorted(contours, key=lambda x: get_square(*x), reverse=True)
    contours = list(filter(lambda x: get_square(*x) > 400, contours))
    if not contours:
        ccc += 1
        continue
    while len(contours) < 4:
        contours = split_rectangles(contours)
    contours = sorted(contours, key=lambda x: x[0][0])
    contours = [expand_rectangle(i, 1.5) for i in contours]
    
    symbols = re.search(r"\\(\w+)\.", all_captchas[idx]).group(1)
    # drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    for i in range(4):
        color = (random.randint(0,256), random.randint(0,256), random.randint(0,256))
        # morph = cv.rectangle(morph, contours[i][0], contours[i][1], color, 2)
        # cv.drawContours(drawing, contours, i, color, 2, cv.LINE_8, hierarchy, 0)
        curr_path = os.path.join(path, str(target_mapper[symbols[i]]))
        curr_path = os.path.join(curr_path, "{0:010d}.png".format(curr_symb))
        curr_symb += 1
        crop_img = morph[contours[i][0][1]:contours[i][1][1], contours[i][0][0]:contours[i][1][0]]
        # morph[contours[i][0][1]:contours[i][1][1], contours[i][0][0]:contours[i][1][0]]
        cv.imwrite(curr_path, crop_img)
        
# cv.imshow("", morph)
# cv.waitKey()
# cv.destroyAllWindows()

In [21]:
# cv.imshow("", img)
# cv.waitKey()
# cv.destroyAllWindows()

In [22]:
# cv.imshow("", morph)
# cv.waitKey()
# cv.destroyAllWindows()

# Model creation

In [133]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [134]:
seed_value= 145
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED']=str(seed_value)

In [135]:
train_path = os.path.join(os.getcwd(), "training_data")
test_path = os.path.join(os.getcwd(), "test_data")

In [136]:
## Set up hyperparameters that will be used later
hyper_dimension = 128
hyper_batch_size = 512
hyper_epochs = 100
hyper_channels = 1
hyper_mode = 'grayscale'

In [137]:
## Generate batches of image data (train, validation, and test) with data augmentation
## Augmentation helps to lower variance of complex model
train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

In [138]:
train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'categorical',
                                                    # classes = target_mapper.values(),
                                                    seed = seed_value)
test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'categorical',
                                                  # classes = target_mapper.values(),
                                                 color_mode = hyper_mode,
                                                 shuffle=True,
                                                 seed = seed_value)

test_generator.reset()

Found 33052 images belonging to 26 classes.
Found 9052 images belonging to 26 classes.


In [139]:
train_generator.image_shape

(128, 128, 1)

In [140]:
from keras.regularizers import l2

In [141]:
# Build the neural network!
model = Sequential()
model.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))

# First convolutional layer with max pooling
model.add(Conv2D(filters=32, kernel_size=3, activation="relu", kernel_regularizer=l2(0.05), bias_regularizer=l2(0.05)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Second convolutional layer with max pooling
model.add(Conv2D(filters=32, kernel_size=3, activation="relu", kernel_regularizer=l2(0.05), bias_regularizer=l2(0.05)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=3, activation="relu", kernel_regularizer=l2(0.05), bias_regularizer=l2(0.05)))
model.add(Flatten())
# Hidden layer with 500 nodes
model.add(Dense(500, activation="relu", kernel_regularizer=l2(0.05), bias_regularizer=l2(0.05)))

# Output layer with 36 nodes (one for each possible letter/number we predict)
model.add(Dense(26, activation="softmax", kernel_regularizer=l2(0.05), bias_regularizer=l2(0.05)))

# Ask Keras to build the TensorFlow model behind the scenes
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [142]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [143]:
hist = model.fit(
    train_generator, 
    steps_per_epoch = len(train_generator),
    epochs = hyper_epochs,
    validation_data = test_generator,
    validation_steps = len(test_generator),
    verbose=2,
    # callbacks=[early_stopping]
)

Epoch 1/100
65/65 - 318s - 5s/step - accuracy: 0.3006 - loss: 9.4715 - val_accuracy: 0.5152 - val_loss: 3.4034
Epoch 2/100
65/65 - 0s - 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
65/65 - 125s - 2s/step - accuracy: 0.5798 - loss: 2.7592 - val_accuracy: 0.6244 - val_loss: 2.3454
Epoch 4/100
65/65 - 0s - 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
65/65 - 122s - 2s/step - accuracy: 0.6417 - loss: 2.1412 - val_accuracy: 0.6378 - val_loss: 2.0280
Epoch 6/100
65/65 - 0s - 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
65/65 - 123s - 2s/step - accuracy: 0.6700 - loss: 1.9186 - val_accuracy: 0.6826 - val_loss: 1.8415
Epoch 8/100
65/65 - 0s - 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/100
65/65 - 121s - 2s/step - accuracy: 0.693

# Save the model to binary

In [144]:
model.save('cnn_captcha.h5')

# Prediction of model

In [145]:
back_map = {}
for index, value in enumerate(ALL_SYMBOLS):
    back_map[str(index)] = value

In [146]:
classes = list(train_generator.class_indices.keys())

In [147]:
back_map

{'0': 'A',
 '1': 'B',
 '2': 'C',
 '3': 'D',
 '4': 'E',
 '5': 'F',
 '6': 'G',
 '7': 'H',
 '8': 'I',
 '9': 'J',
 '10': 'K',
 '11': 'L',
 '12': 'M',
 '13': 'N',
 '14': 'O',
 '15': 'P',
 '16': 'Q',
 '17': 'R',
 '18': 'S',
 '19': 'T',
 '20': 'U',
 '21': 'V',
 '22': 'W',
 '23': 'X',
 '24': 'Y',
 '25': 'Z'}

In [148]:
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
m = load_model('cnn_captcha.h5')

# Print the model summary
m.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 500)                 │      12,544,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 26)                  │          13,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,576,344 (47.97 MB)

 Trainable params: 12,576,342 (47.97 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [162]:
def preprocess():

    pathes = []
    curr_symb = 0
    img = cv.imread("test.png")
    
    # convert to grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # blur
    blur = cv.GaussianBlur(gray, (0, 0), sigmaX=33, sigmaY=33)
    # divide
    divide = cv.divide(gray, blur, scale=255)
    # otsu threshold
    thresh = cv.threshold(divide, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)[1] # 
    # apply morphology
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (4, 4))
    morph = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)

    threshold = 150
    canny_output = cv.Canny(morph, threshold, threshold * 2)
    canny_output_cp = canny_output.copy()
    canny_output = cv.blur(canny_output, (3, 3))
    
    # cv.imshow("", canny_output)
    # cv.waitKey()
    # cv.destroyAllWindows()
    
    contours, hierarchy = cv.findContours(canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = [get_corners(i) for i in contours]
    contours = filter_inner_contour(contours)
    contours = sorted(contours, key=lambda x: get_square(*x), reverse=True)
    contours = list(filter(lambda x: get_square(*x) > 400, contours))
    if not contours:
        return pathes
    while len(contours) < 4:
        contours = split_rectangles(contours)
    contours = sorted(contours, key=lambda x: x[0][0])
    contours = [expand_rectangle(i, 1.5) for i in contours]
    
    # symbols = re.search(r"\\(\w+)\.", all_captchas[idx]).group(1)
    # drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    for i in range(4):
        color = (random.randint(0,256), random.randint(0,256), random.randint(0,256))
        # morph = cv.rectangle(morph, contours[i][0], contours[i][1], color, 2)
        # cv.drawContours(drawing, contours, i, color, 2, cv.LINE_8, hierarchy, 0)
        # curr_path = os.path.join(os.getcwd(), str(target_mapper[symbols[i]]))
        curr_path = os.path.join(os.getcwd(), "{0:010d}.png".format(curr_symb))
        curr_symb += 1
        crop_img = morph[contours[i][0][1]:contours[i][1][1], contours[i][0][0]:contours[i][1][0]]
        # morph[contours[i][0][1]:contours[i][1][1], contours[i][0][0]:contours[i][1][0]]
        cv.imwrite(curr_path, crop_img)
        pathes.append(curr_path)

    return pathes

In [163]:
def gen_captcha_text(length=4):

    symbols_list = []
    for _ in range(length):
        symbols_list.append(random.choice(ALL_SYMBOLS))

    return "".join(symbols_list)

In [267]:
from captcha.image import ImageCaptcha

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [294]:
def run_test():
    image = ImageCaptcha(width = 280, height = 90)
    captcha_text = gen_captcha_text(length=4)
    # generate the image of the given text
    data = image.generate(captcha_text)  
    # write the image on the given file and save it
    image.write(captcha_text, f'test.png')

    captcha_ls = []
    for path in preprocess():
        img = cv.imread(path)
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
        # Resize the image to the target input shape (e.g., 64x64 pixels)
        target_size = (hyper_dimension, hyper_dimension)
        img = cv.resize(img, target_size)
    
        # cv.imshow("", img)
        # cv.waitKey()
        # cv.destroyAllWindows()
        
        # Convert the resized image to a NumPy array
        img_array = img_to_array(img)
        
        # Expand the dimensions of the image array to match the expected input shape of the model
        img_input = np.expand_dims(img_array, axis=0)
        
        # Normalize the image data to a range of [0, 1] (optional, depending on model requirements)
        img_input = img_input / 255.0  # Assuming pixel values are in the range [0, 255]
        predictions = m.predict(img_input, verbose=0)
        predictions = predictions[0]
    
        captcha_ls.append(back_map[classes[predictions.argmax()]])
        
    result = "".join(captcha_ls)
    print(f"true_value: {captcha_text}\tpredicted_value: {result}")

In [296]:
for _ in range(50):
    run_test()

true_value: ENFB	predicted_value: ENFB
true_value: ZMXI	predicted_value: ZMXI
true_value: BHIR	predicted_value: BHTK
true_value: KBRE	predicted_value: KBRE
true_value: HIAX	predicted_value: HIAX
true_value: OOTA	predicted_value: OOTA
true_value: BEQD	predicted_value: BEQO
true_value: SGKH	predicted_value: SGKH
true_value: ZXMR	predicted_value: IXMR
true_value: HUTD	predicted_value: HQTD
true_value: BNAT	predicted_value: HVAT
true_value: DELK	predicted_value: OELK
true_value: UJPU	predicted_value: UJPU
true_value: FJBJ	predicted_value: FJBJ
true_value: WAQM	predicted_value: WAQM
true_value: VIXP	predicted_value: VIXP
true_value: LJLF	predicted_value: JJJF
true_value: WVHZ	predicted_value: WVHZ
true_value: FKBV	predicted_value: FKBV
true_value: NTOP	predicted_value: NTOP
true_value: BJPM	predicted_value: BJPM
true_value: ZFSH	predicted_value: ZRSH
true_value: TGMW	predicted_value: TGMW
true_value: KAXZ	predicted_value: KAXZ
true_value: XMOM	predicted_value: KMOM
true_value: IHZZ	predicte

In [297]:
0.8154**4

0.4420616378896656